# Imports

In [ ]:
try:
    from requests_html import HTMLSession
except: 
    !pip install requests_html
    from requests_html import HTMLSession
    
try:
    import pandas as pd
    import numpy as np
except:
    !pip install pandas
    import pandas as pd
    import numpy as np

try:
    import plotly.graph_objects as go
    import plotly.subplots as ms
except:
    !pip install plotly
    import plotly.graph_objects as go
    import plotly.subplots as ms
    
try:
    from datetime import datetime
except:
    !pip install datetime
    from datetime import datetime
    
try:
    import time
except:
    !pip install time
    import time
    
try:
    import requests
except:
    !pip install requests
    import requests

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import os.path

import warnings
warnings.filterwarnings('ignore')

# Functions

In [ ]:
def market_info(market):
    market = market.replace('(', '')
    market = market.replace(')', '')
    market = market.replace(' ', '-')

    url = 'https://www.ingsprinters.nl/sprinters/' + market

    r = HTMLSession().get(url)

    value = r.html.find('span', clean=True)
    
    return float(value[4].text.strip().replace('.', '').replace(',', '.'))

In [ ]:
def csv_log_file(ticker, df):
    if os.path.isfile('../data/' + ticker + '_log.csv'):
        df.to_csv('../data/' + ticker + '_log.csv',
                       sep=';',
                       decimal=',',
                       index=False)
    else:
        df.to_csv('../data/' + ticker + '_log.csv',
                  sep=';',
                  decimal=',',
                  index=False)

In [ ]:
def find_pattern(df, pattern_list, name='all'):
    all_pattern_data = {}

    open = df['open'].values
    high = df['high'].values
    low = df['low'].values
    close = df['close'].values

    open_h1 = df['open'].shift(1).values
    high_h1 = df['high'].shift(1).values
    low_h1 = df['low'].shift(1).values
    close_h1 = df['close'].shift(1).values

    open_h2 = df['open'].shift(2).values
    high_h2 = df['high'].shift(2).values
    low_h2 = df['low'].shift(2).values
    close_h2 = df['close'].shift(2).values

    temp = {}
    for i in pattern_list:
        temp[i] = is_pattern(i, [open, high, low, close],
                             [open_h1, high_h1, low_h1, close_h1],
                             [open_h2, high_h2, low_h2, close_h2])
    drop_list = []

    for i in temp.keys():
        x = 0
        for j in range(len(temp[i])):
            if temp[i][j] == False:
                x = x + 1
        if x == 6:
            drop_list = drop_list + [i]
    
    result=pd.DataFrame(temp).drop(drop_list, axis = 1)

    return result

In [ ]:
def INGSprinter_candlestick(ticker, refreshtime, interval, file_mgmt,
                            candlestick, candlestick_timer, pattern_list):
    i = 0
    if file_mgmt and os.path.isfile('../data/' + ticker + '_log.csv'):
        df = pd.read_csv('../data/' + ticker + '_log.csv',
                         sep=';',
                         decimal=',')
        df = df[df.Time >= datetime.today().strftime('%Y-%m-%d')]
    else:
        df = pd.DataFrame(columns=['Time', 'Rate'])
    while i < refreshtime / interval:
        a = datetime.now()

        df = df.append({
            'Time': datetime.now(),
            'Rate': market_info(ticker)
        },
                       ignore_index=True)

        print(market_info(ticker), datetime.now())

        if candlestick:
            df_copy = df.copy()
            df_copy['Time'] = pd.to_datetime(df_copy['Time'])
            df_copy = df_copy.set_index('Time')
            df_new = df_copy['Rate'].resample(candlestick_timer).ohlc()

            #check if df_new has at least 6 rows
            if df_new.shape[0] > 5:
                result = find_pattern(df_new.dropna().tail(6), pattern_list)
                display(result.tail(3).style.format("{}"))


            fig = go.Figure(data=[
                go.Candlestick(x=df_new.index,
                               open=df_new['open'],
                               high=df_new['high'],
                               low=df_new['low'],
                               close=df_new['close'])
            ])
            fig.show()

        else:
            df.plot(x='Time', y=['Rate'], figsize=(16, 9))
            plt.show()

        b = datetime.now()

        if interval - (b - a).total_seconds() > 0:
            time.sleep(interval - (b - a).total_seconds())
        if file_mgmt:
            csv_log_file(ticker, df)

        clear_output(wait=True)
        i = i + 1

In [ ]:
import numpy as np

np.seterr(divide='ignore', invalid='ignore')


def is_pattern(pattern, ohlc, ohlc_h1, ohlc_h2):
    open = ohlc[0]
    high = ohlc[1]
    low = ohlc[2]
    close = ohlc[3]

    open_h1 = ohlc_h1[0]
    high_h1 = ohlc_h1[1]
    low_h1 = ohlc_h1[2]
    close_h1 = ohlc_h1[3]

    open_h2 = ohlc_h2[0]
    high_h2 = ohlc_h2[1]
    low_h2 = ohlc_h2[2]
    close_h2 = ohlc_h2[3]

    if pattern == 'doji':
        return (abs(close - open) / (high - low) < 0.05) & \
        (4*(high - np.maximum(close, open)) > (high - low)) & \
        (4*(np.minimum(close, open) - low) > (high - low)) & \
        (high > low)
    else:
        pass

    if pattern == 'gravestone_doji':
        return (abs(close - open) / (high - low) < 0.05) & \
        (5*(high - np.maximum(close, open)) > (high - low)) & \
        (5*(np.minimum(close, open) - low) < (high - low)) & \
        (high > low)
    else:
        pass

    if pattern == 'dragonfly_doji':
        return (abs(close - open) / (high - low) < 0.05) & \
        (5*(high - np.maximum(close, open)) < (high - low)) & \
        (5*(np.minimum(close, open) - low) > (high - low)) & \
        (high > low)
    else:
        pass

    if pattern == 'bullish_spinning_top':
        return (abs(close - open) / (high - low) > 0.1) & \
        (abs(close - open) / (high - low) < 0.2) & \
        (5*(high - np.maximum(close, open)) > (high - low)) & \
        (5*(np.minimum(close, open) - low) > (high - low)) & \
        (close > open)
    else:
        pass

    if pattern == 'bearish_spinning_top':
        return (abs(close - open) / (high - low) > 0.1) & \
        (abs(close - open) / (high - low) < 0.2) & \
        (5*(high - np.maximum(close, open)) > (high - low)) & \
        (5*(np.minimum(close, open) - low) > (high - low)) & \
        (close < open)
    else:
        pass

    if pattern == 'inverted_Hammer':
        return (abs(close - open) / (high - low) > 0.1) & \
        (abs(close - open) / (high - low) < 0.3) & \
        (5*(high - np.maximum(close, open)) > (high - low)) & \
        (10*(np.minimum(close, open) - low) < (high - low)) & \
        (close > open)
    else:
        pass

    if pattern == 'shootingstar':
        return (abs(close - open) / (high - low) > 0.1) & \
        (abs(close - open) / (high - low) < 0.3) & \
        (5*(high - np.maximum(close, open)) > (high - low)) & \
        (10*(np.minimum(close, open) - low) < (high - low)) & \
        (close < open)
    else:
        pass

    if pattern == 'hanging_man':
        return (abs(close - open) / (high - low) > 0.1) & \
        (abs(close - open) / (high - low) < 0.3) & \
        (10*(high - np.maximum(close, open)) < (high - low)) & \
        (5*(np.minimum(close, open) - low) > (high - low)) & \
        (close < open)
    else:
        pass

    if pattern == 'hammer':
        return (abs(close - open) / (high - low) > 0.1) & \
        (abs(close - open) / (high - low) < 0.3) & \
        (10*(high - np.maximum(close, open)) < (high - low)) & \
        (5*(np.minimum(close, open) - low) > (high - low)) & \
        (close > open)
    else:
        pass

    if pattern == 'bullish_marubozu':
        return (abs(close - open) >= 0.95*(high - low))& \
        (close > open)
    else:
        pass

    if pattern == 'bearish_marubozu':
        return (abs(close - open) >= 0.95*(high - low))& \
        (close < open)
    else:
        pass

    if pattern == 'bullish_belt_hold':
        return (abs(close - open) >= 0.90*(high - low))& \
        (abs(open - low)*100 < (high - low) ) & \
        (high > close) & \
        (close > open)
    else:
        pass

    if pattern == 'bearish_belt_hold':
        return (abs(close - open) >= 0.90*(high - low))& \
        (abs(open - high)*100 < (high - low)) & \
        (low < close) & \
        (close < open)
    else:
        pass

# Double

    if pattern == 'bullish_engulfing':
        return (open_h1 > close_h1) & \
        (close > open) & \
        (open_h1 < close) & \
        (close_h1 > open) & \
        (high_h1 < high) & \
        (low_h1 > low)
    else:
        pass

    if pattern == 'bearish_engulfing':
        return (open_h1 < close_h1) & \
        (close < open) & \
        (open_h1 > close) & \
        (close_h1 < open) & \
        (high_h1 < high) & \
        (low_h1 > low)
    else:
        pass

    if pattern == 'tweezer_top':
        return (open_h1 < close_h1) & \
        (close < open) & \
        (20*abs(high_h1 - high) < (high - low))
    else:
        pass

    if pattern == 'tweezer_bottom':
        return (open_h1 > close_h1) & \
        (close > open) & \
        (20*abs(low_h1 - low) < (high - low))
    else:
        pass

    if pattern == 'bullish_separating_line':
        return (open_h1 < open) & \
        (open_h1 > close_h1) & \
        (open < close)
    else:
        pass

    if pattern == 'bearish_separating_line':
        return (open_h1 > open) & \
        (open_h1 < close_h1) & \
        (open > close)
    else:
        pass

    if pattern == 'piercing_line':
        return (open_h1 > close_h1) & \
        (open < close) & \
        (close_h1 > open) & \
        (open_h1 > close) & \
        ((open_h1 - close_h1) < 2*(close - close_h1))
    else:
        pass

    if pattern == 'dark_cloud_cover':
        return (open_h1 < close_h1) & \
        (open > close) & \
        (close_h1 < open) & \
        (open_h1 < close) & \
        ((close_h1 - open_h1) < 2*(close_h1 - close))
    else:
        pass

    if pattern == 'mathing_high':
        return (open_h1 < close_h1) & \
        (close > open) & \
        (20*abs(high_h1 - high) < (high - low))
    else:
        pass

    if pattern == 'mathing_low':
        return (open_h1 > close_h1) & \
        (close < open) & \
        (20*abs(low_h1 - low) < (high - low))
    else:
        pass

    if pattern == 'bullish_harami':
        return (open_h1 > close_h1) &\
        (close > open) &\
        (open_h1 > close) &\
        (close_h1  < open) &\
        ((open_h1 - close_h1) > 4*(close - open))
    else:
        pass

    if pattern == 'bearish_harami':
        return (open_h1 < close_h1) &\
        (close < open) &\
        (open_h1 < close) &\
        (close_h1 > open) &\
        ((close_h1 - open_h1) > 4*(open - close))
    else:
        pass

    if pattern == 'bullish_harami_cross':
        return (open_h1 > close_h1) & \
        (open_h1 > close) & \
        (close_h1  < open) & \
        (abs(close - open) / (high - low) < 0.05) & \
        (4*(high - np.maximum(close, open)) > (high - low)) & \
        (4*(np.minimum(close, open) - low) > (high - low))
    else:
        pass

    if pattern == 'bearish_harami_cross':
        return (open_h1 < close_h1) & \
        (open_h1 < close) & \
        (close_h1  > open) & \
        (abs(close - open) / (high - low) < 0.05) & \
        (4*(high - np.maximum(close, open)) > (high - low)) & \
        (4*(np.minimum(close, open) - low) > (high - low))
    else:
        pass

    if pattern == 'descending_hawk':
        return (open_h1 < close_h1) & \
        (close > open) & \
        (open_h1 < open) & \
        (close_h1  > close)
    else:
        pass

    if pattern == 'homing_pigeon':
        return (open_h1 > close_h1) & \
        (close < open) & \
        (open_h1 > open) & \
        (close_h1 < close)
    else:
        pass

    if pattern == 'bullish_in_neck':
        return (open_h1 < close_h1) & \
        (close < open) & \
        (20*abs(close_h1 - close) < (high - low_h1))
    else:
        pass

    if pattern == 'bearish_in_neck':
        return (open_h1 > close_h1) & \
        (close > open) & \
        (20*abs(close_h1 - close) < (high_h1 - low))
    else:
        pass

    if pattern == 'bullish_on_neck':
        return (open_h1 < close_h1) & \
        (close < open) & \
        (20*abs(high_h1 - low) < (high - low_h1))
    else:
        pass

    if pattern == 'bearish_on_neck':
        return (open_h1 > close_h1) & \
        (close > open) & \
        (20*abs(low_h1 - high) < (high_h1 - low))
    else:
        pass

    if pattern == 'bullish_kicking':
        return (open_h1 > close_h1) & \
        (open < close) & \
        ((open_h1 - close_h1) > 0.9*(high_h1 - low_h1)) & \
        ((close_h1 - open_h1) > 0.9*(high - low)) & \
        (open_h1 < open)
    else:
        pass

    if pattern == 'bearish_kicking':
        return (open_h1 < close_h1) & \
        (open > close) & \
        ((close_h1 - open_h1) > 0.9*(high_h1 - low_h1)) & \
        ((open - close) > 0.9*(high - low)) & \
        (open_h1 > open)
    else:
        pass


#triple

    if pattern == 'morning_star':
        return (open_h2 > close_h2) &\
        (3*abs(close_h1 - open_h1) < abs(close_h2 - open_h2)) &\
        (3*abs(close_h1 - open_h1) < abs(close - open)) &\
        (np.maximum(open_h1, close_h1) < close_h2) &\
        (np.maximum(open_h1, close_h1) < open) &\
        (high_h1 > low_h1) &\
        (close > open)
    else:
        pass

    if pattern == 'evening_star':
        return (open_h2 < close_h2) &\
        (3*abs(close_h1 - open_h1) < abs(close_h2 - open_h2)) &\
        (3*abs(close_h1 - open_h1) < abs(close - open)) &\
        (np.minimum(open_h1, close_h1) > close_h2) &\
        (np.minimum(open_h1, close_h1) > open) &\
        (high_h1 > low_h1) &\
        (close < open)
    else:
        pass

    if pattern == 'morning_doji_star':
        return (open_h2 > close_h2) &\
        (abs(close_h1 - open_h1)/(high_h1 - low_h1) < 0.05) &\
        (np.minimum(open_h1, close_h1) < close_h2) &\
        (np.minimum(open_h1, close_h1) < open) &\
        (close > open)
    else:
        pass

    if pattern == 'evening_doji_star':
        return (open_h2 < close_h2) &\
        (abs(close_h1 - open_h1)/(high_h1 - low_h1) < 0.05) &\
        (np.minimum(open_h1, close_h1) > close_h2) &\
        (np.minimum(open_h1, close_h1) > open) &\
        (close < open)
    else:
        pass

    if pattern == 'three_white_soldier':
        return (open_h2 < close_h2) &\
        (2*abs(close_h2 - open_h2) > (high_h2 - low_h2)) &\
        (open_h1 < close_h1) &\
        (2*abs(close_h1 - open_h1) > (high_h1 - low_h1)) &\
        (open < close) &\
        (2*abs(close - open) > (high - low)) &\
        (0.5*(close_h2 + open_h2) < 0.5*(close_h1 + open_h1)) &\
        (0.5*(close_h1 + open_h1) < 0.5*(close + open)) &\
        (close_h1 > close_h2) &\
        (close > close_h1)
    else:
        pass

    if pattern == 'three_black_crow':
        return (open_h2 > close_h2) &\
        (2*abs(close_h2 - open_h2) > (high_h2 - low_h2)) &\
        (open_h1 > close_h1) &\
        (2*abs(close_h1 - open_h1) > (high_h1 - low_h1)) &\
        (open > close) &\
        (2*abs(close - open) > (high - low)) &\
        (0.5*(close_h2 + open_h2) > 0.5*(close_h1 + open_h1)) &\
        (0.5*(close_h1 + open_h1) > 0.5*(close + open)) &\
        (close_h1 < close_h2) &\
        (close < close_h1)
    else:
        pass

    if pattern == 'three_inside_up':
        return (open_h2 > close_h2) &\
        (open_h1 < close_h1) &\
        (open < close) &\
        (open_h1 < open) &\
        ((open + close) > (open_h1 + close_h1)) &\
        (open_h2 > close_h1) &\
        (close_h2 < open_h1)
    else:
        pass

    if pattern == 'three_inside_down':
        return (open_h2 < close_h2) &\
        (open_h1 > close_h1) &\
        (open > close) &\
        (open_h1 > open) &\
        ((open + close) < (open_h1 + close_h1))&\
        (close_h2 > open_h1) &\
        (open_h2 < close_h1)
    else:
        pass

    if pattern == 'deliberation':
        return (open_h2 < close_h2) &\
        (2*abs(close_h2 - open_h2) > (high_h2 - low_h2)) &\
        (open_h1 < close_h1) &\
        (2*abs(close_h1 - open_h1) > (high_h1 - low_h1)) &\
        (open < close) &\
        (3*abs(close - open) < abs(close_h1 - open_h1)) &\
        (0.5*(close_h2 + open_h2) < 0.5*(close_h1 + open_h1)) &\
        (0.5*(close_h1 + open_h1) < 0.5*(close + open)) &\
        (close_h1 > close_h2) &\
        (close > close_h1)
    else:
        pass

    if pattern == 'three_outside_up':
        return (open_h2 > close_h2) &\
        (open_h1 < close_h1) &\
        (open < close) &\
        (open_h1 < open) &\
        ((open + close) > (open_h1 + close_h1)) &\
        (open_h2 < close_h1) &\
        (close_h2 > open_h1)
    else:
        pass

    if pattern == 'three_outside_down':
        return (open_h2 < close_h2) &\
        (open_h1 > close_h1) &\
        (open > close) &\
        (open_h1 > open) &\
        ((open + close) < (open_h1 + close_h1))&\
        (close_h2 < open_h1) &\
        (open_h2 > close_h1)
    else:
        pass

    if pattern == 'bullish_abandoned_baby':
        return (open_h2 > close_h2) &\
        (3*abs(close_h1 - open_h1) < abs(close_h2 - open_h2)) &\
        (3*abs(close_h1 - open_h1) < abs(close - open)) &\
        (np.maximum(open_h1, close_h1) < close_h2) &\
        (np.maximum(open_h1, close_h1) < open) &\
        (high_h1 > low_h1) &\
        (high_h1 < low_h2) &\
        (high_h1 < low) &\
        (close > open)
    else:
        pass

    if pattern == 'bearish_abandoned_baby':
        return (open_h2 < close_h2) &\
        (3*abs(close_h1 - open_h1) < abs(close_h2 - open_h2)) &\
        (3*abs(close_h1 - open_h1) < abs(close - open)) &\
        (np.minimum(open_h1, close_h1) > close_h2) &\
        (np.minimum(open_h1, close_h1) > open) &\
        (high_h1 > low_h1) &\
        (low_h1 > high_h2) &\
        (low_h1 > high) &\
        (close < open)
    else:
        pass

    if pattern == 'bullish_stick_sandwich':
        return (open_h2 > close_h2) &\
        (open_h1 < close_h1) &\
        (open > close) &\
        (open_h1 > close_h2) &\
        (open_h2 < close_h1) &\
        (open > close_h1) &\
        (close < open_h1) &\
        (2*(open_h2 - close_h2) > (high_h2 - low_h2)) &\
        (2*(open - close) > (high - low)) &\
        (20*abs(close_h2 - close) < (high - np.minimum(low_h2,low)))
    else:
        pass

    if pattern == 'bearish_stick_sandwich':
        return (open_h2 < close_h2) &\
        (open_h1 > close_h1) &\
        (open < close) &\
        (open_h1 < close_h2) &\
        (open_h2 > close_h1) &\
        (open < close_h1) &\
        (close > open_h1) &\
        (2*(close_h2 - open_h2) > (high_h2 - low_h2)) &\
        (2*(close - open) > (high - low)) &\
        (20*abs(close_h2 - close) < (low - np.maximum(high_h2, high)))
    else:
        pass

    if pattern == 'bullish_side_by_side_white_line':
        return (open_h2 < close_h2) &\
        (open_h1 < close_h1) &\
        (open < close) &\
        (2*(close_h2 - open_h2) > (high_h2 - low_h2)) &\
        (2*(close_h1 - open_h1) > (high_h1 - low_h1)) &\
        (2*(close - open) > (high - low)) &\
        (open_h1 > close_h2) &\
        (20*abs(close_h1 - close) < (np.maximum(high, high_h1)-np.minimum(low, low_h1))) &\
        (20*abs(open_h1 - open) < (np.maximum(high, high_h1)-np.minimum(low, low_h1)))
    else:
        pass

    if pattern == 'bearish_side_by_side_black_line':
        return (open_h2 > close_h2) &\
        (open_h1 > close_h1) &\
        (open > close) &\
        (2*(open_h2 - close_h2) > (high_h2 - low_h2)) &\
        (2*(open_h1 - close_h1) > (high_h1 - low_h1)) &\
        (2*(open - close) > (high - low)) &\
        (open_h1 < close_h2) &\
        (20*abs(close_h1 - close) < (np.maximum(high, high_h1)-np.minimum(low, low_h1))) &\
        (20*abs(open_h1 - open) < (np.maximum(high, high_h1)-np.minimum(low, low_h1)))
    else:
        pass

    if pattern == 'bearish_side_by_side_white_line':
        return (open_h2 > close_h2) &\
        (open_h1 < close_h1) &\
        (open < close) &\
        (2*(open_h2 - close_h2) > (high_h2 - low_h2)) &\
        (2*(close_h1 - open_h1) > (high_h1 - low_h1)) &\
        (2*(close - open) > (high - low)) &\
        (close_h1 < close_h2) &\
        (20*abs(close_h1 - close) < (np.maximum(high, high_h1)-np.minimum(low, low_h1))) &\
        (20*abs(open_h1 - open) < (np.maximum(high, high_h1)-np.minimum(low, low_h1)))
    else:
        pass

    if pattern == 'bullish_side_by_side_black_line':
        return (open_h2 < close_h2) &\
        (open_h1 > close_h1) &\
        (open > close) &\
        (2*(close_h2 - open_h2) > (high_h2 - low_h2)) &\
        (2*(open_h1 - close_h1) > (high_h1 - low_h1)) &\
        (2*(open - close) > (high - low)) &\
        (close_h1 > close_h2) &\
        (20*abs(close_h1 - close) < (np.maximum(high, high_h1)-np.minimum(low, low_h1))) &\
        (20*abs(open_h1 - open) < (np.maximum(high, high_h1)-np.minimum(low, low_h1)))
    else:
        pass

    if pattern == 'upside_gap_three':
        return (open_h2 < close_h2) &\
        (open_h1 < close_h1) &\
        (open > close) &\
        (close_h2 < open_h1) &\
        (open > open_h1) &\
        (close < close_h2)
    else:
        pass

    if pattern == 'downside_gap_three':
        return (open_h2 > close_h2) &\
        (open_h1 > close_h1) &\
        (open < close) &\
        (close_h2 > open_h1) &\
        (open < open_h1) &\
        (close > close_h2)
    else:
        pass

    if pattern == 'upside_gap_two_crow':
        return (open_h2 < close_h2) &\
        (open_h1 > close_h1) &\
        (open > close) &\
        (open_h1 < open) &\
        (close_h1 > close) &\
        (close_h2 < close)
    else:
        pass

    if pattern == 'unique_three_river_bottom':
        return (open_h2 > close_h2) &\
        (open_h1 > close_h1) &\
        (open < close) &\
        ((close_h1 - low_h1) > 2*(open_h1 - close_h1)) &\
        (close < close_h1)&\
        (open > low_h1)
    else:
        pass

    if pattern == 'bullish_tri_star':
        return (abs(close - open) / (high - low) < 0.05) & \
        (4*(high - np.maximum(close, open)) > (high - low)) & \
        (4*(np.minimum(close, open) - low) > (high - low)) & \
        (high > low)& \
        (abs(close_h1 - open_h1) / (high_h1 - low_h1) < 0.05) & \
        (4*(high_h1 - np.maximum(close_h1, open_h1)) > (high_h1 - low_h1)) & \
        (4*(np.minimum(close_h1, open_h1) - low_h1) > (high_h1 - low_h1)) & \
        (high_h1 > low_h1)& \
        (abs(close_h2 - open_h2) / (high_h2 - low_h2) < 0.05) & \
        (4*(high_h2 - np.maximum(close_h2, open_h2)) > (high_h2 - low_h2)) & \
        (4*(np.minimum(close_h2, open_h2) - low_h2) > (high_h2 - low_h2)) & \
        (high_h2 > low_h2) & \
        (close_h2 > close_h1) & \
        (close > close_h1)
    else:
        pass

    if pattern == 'bearish_tri_star':
        return (abs(close - open) / (high - low) < 0.05) & \
        (4*(high - np.maximum(close, open)) > (high - low)) & \
        (4*(np.minimum(close, open) - low) > (high - low)) & \
        (high > low)& \
        (abs(close_h1 - open_h1) / (high_h1 - low_h1) < 0.05) & \
        (4*(high_h1 - np.maximum(close_h1, open_h1)) > (high_h1 - low_h1)) & \
        (4*(np.minimum(close_h1, open_h1) - low_h1) > (high_h1 - low_h1)) & \
        (high_h1 > low_h1)& \
        (abs(close_h2 - open_h2) / (high_h2 - low_h2) < 0.05) & \
        (4*(high_h2 - np.maximum(close_h2, open_h2)) > (high_h2 - low_h2)) & \
        (4*(np.minimum(close_h2, open_h2) - low_h2) > (high_h2 - low_h2)) & \
        (high_h2 > low_h2) & \
        (close_h2 < close_h1) & \
        (close < close_h1)
    else:
        pass

    if pattern == 'three_star_in_the_north':
        return (close_h2 > open_h2) & \
        (close_h1 > open_h1) & \
        (close > open) & \
        (close_h2 - open_h2 > close_h1 - open_h1) & \
        (close_h1 - open_h1 > close - open) & \
        (close_h2 < close_h1) & \
        (close_h1 < close)
    else:
        pass

    if pattern == 'three_star_in_the_south':
        return (close_h2 < open_h2) & \
        (close_h1 < open_h1) & \
        (close < open) & \
        (close_h2 - open_h2 < close_h1 - open_h1) & \
        (close_h1 - open_h1 < close - open) & \
        (close_h2 > close_h1) & \
        (close_h1 > close)
    else:
        pass

# Settings

In [ ]:
single_pattern = [
    'doji', 'gravestone_doji', 'dragonfly_doji', 'bullish_spinning_top',
    'bearish_spinning_top', 'inverted_Hammer', 'shootingstar', 'hanging_man',
    'hammer', 'bullish_marubozu', 'bearish_marubozu', 'bullish_belt_hold',
    'bearish_belt_hold'
]
double_pattern = [
    'bullish_engulfing',
    'bearish_engulfing',
    'tweezer_top',
    'tweezer_bottom',
    'bullish_separating_line',
    'bullish_separating_line',
    'piercing_line',
    'dark_cloud_cover',
    'bullish_harami',  #'matching_high', 'matching_low',
    'bearish_harami',
    'bullish_harami_cross',
    'bearish_harami_cross',
    'descending_hawk',
    'homing_pigeon',
    'bullish_in_neck',
    'bearish_in_neck',
    'bullish_on_neck',
    'bearish_on_neck',
    'bullish_kicking',
    'bearish_kicking'
]
triple_pattern = [
    'morning_star', 'evening_star', 'morning_doji_star', 'evening_doji_star',
    'three_white_soldier', 'three_black_crow', 'three_inside_up',
    'three_inside_down', 'deliberation', 'three_outside_up',
    'three_outside_down', 'bullish_abandoned_baby', 'bearish_abandoned_baby',
    'bullish_stick_sandwich', 'bearish_stick_sandwich',
    'bullish_side_by_side_white_line', 'bearish_side_by_side_black_line',
    'bearish_side_by_side_white_line', 'bullish_side_by_side_black_line',
    'upside_gap_three', 'downside_gap_three', 'upside_gap_two_crow',
    'unique_three_river_bottom', 'bullish_tri_star', 'bearish_tri_star',
    'three_star_in_the_north', 'three_star_in_the_south'
]

pattern_list = single_pattern + double_pattern + triple_pattern

In [ ]:
interval=10
refreshtime=8*60*60
ticker='DAX'
candlestick=True
candlestick_timer='15Min'
if get_ipython(
).__class__.__name__ == 'ZMQInteractiveShell':  #Check if the code is running in Jupyter Notebook
    file_mgmt=True
else: 
    file_mgmt=False

# Run!

In [ ]:
INGSprinter_candlestick(ticker, refreshtime, interval, file_mgmt, candlestick,
                        candlestick_timer, pattern_list)

# Next Steps

In [ ]:
#macd, stochastic, my live chart without volume! 